# Clean Data Extracted from VertNet

In [1]:
import sys
sys.path.append('..')

In [2]:
import re
from pathlib import Path
from pprint import pp
import json
from collections import defaultdict
from ipywidgets import interact
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
DATA_DIR = Path('..') / 'data' / 'output'
ARGS_DIR =  Path('..') / 'args'

BASE_NAME = 'bats_2021-10-19'
VER = 'a'
JSONL = DATA_DIR / 'bats_2021-10-19.jsonl'
OUTPUT = DATA_DIR / f'{BASE_NAME}{VER}.csv'
COUNTS = DATA_DIR / f'{BASE_NAME}{VER}_counts.csv'

ARGS = ARGS_DIR / 'bats.args'

In [4]:
VOCAB_DIR = Path('..') / 'vertnet' / 'vocabulary'

TERMS = VOCAB_DIR / 'life_stage.csv'

## Get traits and other fields from the arguments

In [5]:
TRAITS = [ln.strip().replace('--trait=', '')
          for ln in open(ARGS).readlines() if ln.startswith('--trait=')]
OTHERS = [ln.strip().replace('--extra-field=', '')
          for ln in open(ARGS).readlines() if ln.startswith('--extra-field=')]
# TRAITS
OTHERS

['catalognumber',
 'collectioncode',
 'continent',
 'coordinateprecision',
 'coordinateuncertaintyinmeters',
 'country',
 'countrycode',
 'county',
 'decimallatitude',
 'decimallongitude',
 'eventdate',
 'footprintspatialfit',
 'footprintsrs',
 'footprintwkt',
 'geodeticdatum',
 'geologicalcontextid',
 'georeferencedby',
 'georeferenceddate',
 'georeferenceprotocol',
 'georeferenceremarks',
 'georeferencesources',
 'georeferenceverificationstatus',
 'habitat',
 'highergeography',
 'highergeographyid',
 'highestbiostratigraphiczone',
 'institutioncode',
 'island',
 'islandgroup',
 'lifestage',
 'locality',
 'locationaccordingto',
 'locationid',
 'locationremarks',
 'maximumdepthinmeters',
 'maximumdistanceabovesurfaceinmeters',
 'maximumelevationinmeters',
 'minimumdepthinmeters',
 'minimumdistanceabovesurfaceinmeters',
 'minimumelevationinmeters',
 'municipality',
 'occurrenceid',
 'recordedby',
 'references',
 'scientificname',
 'sex',
 'verbatimcoordinates',
 'verbatimcoordinatesyste

## Trait and field categories

In [6]:
LOC = """ start end field """.split()

MALE = """ testes_size testes_state """.split()

FEMALE = """ embryo_count embryo_length lactation_state
    nipple_count nipple_state ovaries_size ovaries_state
    placental_scar_count pregnancy_state """.split()

REPRO = MALE + FEMALE

BODY = [t for t in TRAITS if t not in REPRO]

GONAD_SIZE = """ testes_size ovaries_size """.split()

## Read data

In [7]:
with open(JSONL) as json_file:
    data = [json.loads(d) for d in json_file.readlines()]
len(data)

72376

In [8]:
data[10]

{'catalognumber': '169358',
 'continent': 'North America',
 'country': 'U S A',
 'county': 'BREWSTER CO',
 'decimallatitude': '29.16',
 'decimallongitude': '-103.61',
 'eventdate': '1974-06-11',
 'geodeticdatum': 'NAD83',
 'georeferencedby': 'Eifler, Maria',
 'georeferenceddate': '2015-03-04',
 'georeferencesources': 'Topoquest.com',
 'locality': 'Big Bend National Park; Terlingua Creek',
 'locationremarks': 'X-site',
 'occurrenceid': '08e7aea6-2340-4eeb-b7cf-e6188871ace0',
 'occurrenceremarks': 'bears Purdue University tag (no catalog number)',
 'recordedby': 'R M  Payne, David Arlen Easterla',
 'sex': 'female',
 'verbatimcoordinatesystem': 'decimal degrees',
 'verbatimeventdate': '1974-06-11',
 'institutioncode': 'KU',
 'collectioncode': 'KUM',
 'dynamicproperties': 'TotalLength: 108.0; TailLength: 35.0; HindFoot: 8.0; EarLength: 13.0; BodyMass: 12.4',
 'scientificname': 'Tadarida brasiliensis mexicana',
 'body_mass': [{'start': 70,
   'end': 84,
   'units': None,
   'value': 12.4,
 

## Normalize Life Stage

In [9]:
terms = pd.read_csv(TERMS).fillna('')

print(terms.shape)
terms.head()

(314, 5)


,label,pattern,attr,replace,notes
0,life_stage,[imm],lower,Juvenile,
1,life_stage,1 mo,lower,Juvenile,
2,life_stage,100% ossified,lower,Adult,
3,life_stage,1st winter,lower,Juvenile,
4,life_stage,1st year,lower,Juvenile,


In [10]:
terms = terms.drop_duplicates()

print(terms.shape)
terms.head()

(314, 5)


,label,pattern,attr,replace,notes
0,life_stage,[imm],lower,Juvenile,
1,life_stage,1 mo,lower,Juvenile,
2,life_stage,100% ossified,lower,Adult,
3,life_stage,1st winter,lower,Juvenile,
4,life_stage,1st year,lower,Juvenile,


In [11]:
terms = terms[terms['replace'] != 'NS']

print(terms.shape)
terms.head()

(314, 5)


,label,pattern,attr,replace,notes
0,life_stage,[imm],lower,Juvenile,
1,life_stage,1 mo,lower,Juvenile,
2,life_stage,100% ossified,lower,Adult,
3,life_stage,1st winter,lower,Juvenile,
4,life_stage,1st year,lower,Juvenile,


In [12]:
# terms.to_csv(TERMS, index=False)

In [13]:
life_stages = {r['pattern']: r['replace'] for _, r in terms.iterrows()}
len(life_stages)

314

In [14]:
for row in data:
    lifestage = row.get('lifestage')
    if lifestage:
        row['lifestage_cor'] = life_stages.get(lifestage, 'NS')

## Get sex notations

In [15]:
sexes = set()
for row in data:
    sex = row.get('sex', '')
    if isinstance(sex, list):
        sex = sex[0]
    if isinstance(sex, dict):
        sex = sex.get('value', '')
    sexes.add(sex)
sexes

{'',
 'f?',
 'female',
 'female ?',
 'female; male',
 'in question',
 'm?',
 'male',
 'male ?',
 'males,females',
 'undetermined',
 'x'}

## Update sex fields

In [16]:
for row in data:
    sex = row.get('sex', '')
    if isinstance(sex, list):
        sex = sex[0]
    if isinstance(sex, dict):
        sex = sex.get('value', '')
    sex = re.split(r'[;,]', sex)
    sex = {s.strip() for s in sex}
    row['sex'] = '; '.join(sex)

## Remove female traits from males and vice versa

In [17]:
for row in data:
    sex = row.get('sex')

    if not sex or ';' in sex or sex[0] not in 'fm':
        continue

    remove = MALE if sex[0] == 'f' else FEMALE
    for key in remove:
        if key in row:
            del row[key]

## Only sex traits in reproductive condition

In [18]:
for row in data:
    for key in BODY:

        if key not in row:
            continue

        # Remove body traits found in reproductive condition
        for trait in row[key]:
            if trait['field'] == 'reproductivecondition':
                del trait

## Get max counts of extracted items

In [19]:
counts = defaultdict(int)
for row in data:
    for field in TRAITS:
        value = row.get(field, {})
        count = 1 if isinstance(value, dict) else len(value)
        counts[field] = max(counts[field], count)

for key, value in counts.items():
    print(key, value)

body_mass 3
ear_length 3
embryo_count 4
embryo_length 4
forearm_length 3
hind_foot_length 3
lactation_state 2
nipple_count 2
nipple_state 2
nipples_enlarged 2
ovaries_size 4
ovaries_state 1
placental_scar_count 2
placental_scar_state 2
pregnancy_state 2
scrotal_state 2
tail_length 4
testes_size 4
testes_state 2
total_length 4
tragus_length 3
vagina_state 2


## Remove traits with too many values

In [20]:
MAX_COUNT = 5

for row in data:
    for trait_name in TRAITS:
        trait = row.get(trait_name)
        if isinstance(trait, list) and len(trait) > MAX_COUNT:
            del row[trait_name]

## Remove empty traits

In [21]:
for row in data:
    for key in TRAITS:
        if key in row and not row[key]:
            del row[key]

## Remove empty records

In [22]:
print(len(data))

data = [d for d in data if any(d.get(t) for t in TRAITS)]

print(len(data))

72376
72210


## Separate gonad length and width measurements

In [23]:
# for row in data:
#     for trait_name in [t for t in GONAD_SIZE if t in row]:
#         for trait in row[trait_name]:
#             if isinstance(trait['value'], list):
#                 trait['length'], trait['width'] = trait['value']
#             elif trait.get('dimension') == 'width':
#                 trait['length'] = ''
#                 trait['width'] = trait['value']
#             else:
#                 trait['length'] = trait['value']
#                 trait['width'] = ''

#             del trait['value']

## Get the maximum gonad length and width measurements

In [24]:
GONAD_SIZE = """ testes_size ovaries_size """.split()

In [25]:
for row in data:

    row['max_testes_length'] = 0
    row['max_testes_width'] = 0
    row['max_ovaries_length'] = 0
    row['max_ovaries_width'] = 0

    for trait_name in [t for t in GONAD_SIZE if t in row]:
        for trait in row[trait_name]:
            gonad = trait_name.split('_')[0]

            len_key = f'max_{gonad}_length'
            wid_key = f'max_{gonad}_width'

            # The trait has both a length and a width
            if isinstance(trait['value'], list):
                length, width = trait['value']
                row[len_key] = max(row[len_key], length)
                row[wid_key] = max(row[wid_key], width)

            # The trait has only a width
            elif trait.get('dimension') == 'width':
                row[wid_key] = max(row[wid_key], width)

            # The trait has only a length
            else:
                row[len_key] = max(row[len_key], length)

    row['max_testes_length'] = row['max_testes_length'] if row['max_testes_length'] else ''
    row['max_testes_width'] = row['max_testes_width'] if row['max_testes_width'] else ''
    row['max_ovaries_length'] = row['max_ovaries_length'] if row['max_ovaries_length'] else ''
    row['max_ovaries_width'] = row['max_ovaries_width'] if row['max_ovaries_width'] else ''

## Combine duplicate extractions

The same trait values may be written in multiple locations or in different fields

In [26]:
for row in data:
    
    for trait_name in [t for t in TRAITS if t in row]:
        
        trait_dict = {}
        
        for trait in row[trait_name]:
        
            loc = {k: v for k, v in trait.items() if k in LOC}
            fld = {k: v for k, v in trait.items() if k not in LOC}

            # Build a key out of the non-location fields
            key = [(k, tuple(v) if isinstance(v, list) else v)
                   for k, v in fld.items()]
            key = tuple(sorted(key))
            
            # Append a location if this is a duplicate
            if key in trait_dict:
                trait_dict[key]['location'].append(loc)

            # Create a new record if this is not a duplicate
            else:
                trait_dict[key] = fld
                trait_dict[key]['location'] = [loc]
        
        # Update the extraction
        values = [v for v in trait_dict.values()]
        row[trait_name] = values[0] if len(values) == 1 else values

## Merge the trait locations into a set

In [27]:
for row in data:
    locations = set()

    for trait_name in [t for t in TRAITS if t in row]:
        if not isinstance(row[trait_name], list):
            row[trait_name] = [row[trait_name]]
        for trait in row[trait_name]:
            for loc in trait['location']:
                locations.add(loc['field'])

    row['loc'] = ' '.join(sorted(locations))

## Remove unwanted columns and values

In [28]:
for row in data:

    for trait_name in [t for t in TRAITS if t in row]:
        
        # Remove gonad sizes because they only want the max values
        if trait_name in GONAD_SIZE:
            del row[trait_name]
        # Just keep the first value? Seems odd to me
        else:
            row[trait_name] = row[trait_name][0]['value']

## Add a binomial to the data

In [29]:
for row in data:
    binomial = row.get('scientificname', '').split()[:2]
    binomial = ' '.join(binomial)
    row['binomial'] = binomial

## Count extracts by binomial

In [30]:
binomials = defaultdict(lambda: defaultdict(int))

for row in data:
    for trait_name in TRAITS:
        if row.get(trait_name):
            binomials[row['binomial']][trait_name] += 1

In [31]:
for binomial, count in binomials.items():
    count['binomial'] = binomial

In [32]:
df = pd.DataFrame(binomials.values()).fillna(0)
df = df.set_index('binomial')
df = df.astype(int)
df = df.sort_index()

In [33]:
df.to_csv(COUNTS)

In [34]:
df.head()

,lactation_state,pregnancy_state,ear_length,hind_foot_length,tail_length,total_length,body_mass,forearm_length,tragus_length,vagina_state,embryo_count,nipple_state,embryo_length,scrotal_state,placental_scar_count,placental_scar_state,testes_state,nipples_enlarged,nipple_count,ovaries_state
binomial,,,,,,,,,,,,,,,,,,,,
Acerodon jubatus,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
Aethalops alecto,0,4,5,5,0,5,5,5,2,0,4,0,4,13,0,0,0,0,0,0
Ametrida centurio,6,24,18,18,0,18,17,1,2,0,33,0,35,10,0,0,1,0,0,0
Ametrida sp.,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Anoura,0,0,5,5,0,5,5,1,1,0,0,0,0,0,0,0,0,0,0,0


## Pivot multiple extractions

In [35]:
pp(data[0])

{'catalognumber': '88708',
 'continent': 'North America',
 'country': 'Canada',
 'county': 'Algoma',
 'decimallatitude': '47.35162',
 'decimallongitude': '-84.63653',
 'eventdate': '1983-07-25',
 'geodeticdatum': 'NAD83',
 'georeferencedby': 'Susan M Woodward (ROM)',
 'georeferenceddate': '20090424',
 'georeferenceprotocol': 'MaNIS/HerpNet/ORNIS Georeferencing Guidelines, GBIF '
                         'Best Practices',
 'georeferencesources': 'Energy, Mines and Resources Canada, Series A751, '
                        'Agawa Bay, 41 N/07, 1:50,000, Edition 3, 1995',
 'georeferenceverificationstatus': 'unverified',
 'highergeography': 'North America; Canada; Ontario; Algoma',
 'lifestage': 'adult',
 'locality': 'Lake Superior Provincial Park, Abandoned Agawa Lodge, Agawa '
             "River Mouth, Trapline 'camp'",
 'locationremarks': 'No UTMG given so determined from topo map',
 'occurrenceid': 'URI:catalog:ROM:Mammals:88708',
 'recordedby': 'Bramwell, RN',
 'reproductivecondition':

In [36]:
# for row in data:
#     for trait_name in [t for t in TRAITS if t in row]:
#         traits = row[trait_name]
#         traits = traits if isinstance(traits, list) else [traits]
#         for i, trait in enumerate(traits, 1):
#             for key, value in trait.items():
#                 new = f'{trait_name}.{i}.{key}'
#                 if key == 'location':
#                     row[new] = json.dumps(value)
#                 elif isinstance(value, list):
#                     row[new] = tuple(value)
#                 else:
#                     row[new] = value
#         del row[trait_name]

## Convert to data frame and remove duplicate records

In [37]:
df = pd.DataFrame(data)
print(df.shape)

# df = df.drop_duplicates()
# print(df.shape)

# columns = sorted(df.columns)
# df = df.reindex(columns, axis='columns').fillna('')
# print(df.shape)

(72210, 74)


In [38]:
df.to_csv(OUTPUT, index=False)